In [ ]:
!pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 106.6 MB/s eta 0:00:00


In [ ]:
import streamlit as st
import pandas as pd
import io
import google.generativeai as genai

# -------------------------
# Configure Gemini
# -------------------------
genai.configure(api_key="")  # Replace with your key
model = genai.GenerativeModel("gemini-1.5-flash")

# -------------------------
# Streamlit UI
# -------------------------
st.title("🧹 AI Data Cleaner using Gemini")

uploaded_file = st.file_uploader("📂 Upload your dataset (CSV or Excel)", type=["csv", "xlsx"])

if uploaded_file:
    # -------------------------
    # Load dataset
    # -------------------------
    if uploaded_file.name.endswith(".csv"):
        df = pd.read_csv(uploaded_file)
    else:
        df = pd.read_excel(uploaded_file)

    st.write("### 🔍 Original Dataset Preview")
    st.dataframe(df.head(10))
    st.write(f"**Original Shape:** {df.shape}")

    # Copy dataset for cleaning
    cleaned_df = df.copy()
    cleaning_steps = []

    # -------------------------
    # 1. Handle duplicates
    # -------------------------
    dup_count = cleaned_df.duplicated().sum()
    if dup_count > 0:
        cleaned_df.drop_duplicates(inplace=True)
        cleaning_steps.append(f"Removed {dup_count} duplicate rows.")

    # -------------------------
    # 2. Handle null values
    # -------------------------
    null_counts = cleaned_df.isnull().sum()
    total_nulls = null_counts.sum()

    if total_nulls > 0:
        for col in cleaned_df.columns:
            if cleaned_df[col].isnull().sum() > 0:
                if cleaned_df[col].dtype in ["int64", "float64"]:
                    mean_val = cleaned_df[col].mean()
                    cleaned_df[col].fillna(mean_val, inplace=True)
                    cleaning_steps.append(
                        f"Filled missing values in **{col}** with mean ({mean_val:.2f})."
                    )
                else:
                    mode_val = cleaned_df[col].mode()[0]
                    cleaned_df[col].fillna(mode_val, inplace=True)
                    cleaning_steps.append(
                        f"Filled missing values in **{col}** with mode ('{mode_val}')."
                    )

    # -------------------------
    # 3. Create cleaning summary prompt
    # -------------------------
    summary_prompt = f"""
    You are an AI Data Cleaner. The dataset had {df.shape[0]} rows and {df.shape[1]} columns.
    After cleaning, it has {cleaned_df.shape[0]} rows and {cleaned_df.shape[1]} columns.

    Cleaning steps performed:
    {cleaning_steps if cleaning_steps else "No cleaning was required."}

    Write a clear, human-like summary report explaining what was cleaned and why.
    """

    # -------------------------
    # 4. Generate Gemini summary
    # -------------------------
    response = model.generate_content(summary_prompt)
    report_text = response.text

    # -------------------------
    # 5. Show results
    # -------------------------
    st.write("### 📝 Cleaning Summary Report")
    st.write(report_text)

    st.write(f"**Cleaned Shape:** {cleaned_df.shape}")

    # -------------------------
    # 6. Download cleaned dataset
    # -------------------------
    output = io.BytesIO()
    cleaned_df.to_csv(output, index=False)
    st.download_button(
        label="⬇️ Download Cleaned Dataset",
        data=output.getvalue(),
        file_name="cleaned_dataset.csv",
        mime="text/csv"
    )

    # -------------------------
    # 7. Download cleaning report
    # -------------------------
    st.download_button(
        label="⬇️ Download Cleaning Report",
        data=report_text,
        file_name="cleaning_report.txt",
        mime="text/plain"
    )


2025-08-29 04:03:03.573 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 04:03:03.899 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-08-29 04:03:03.900 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 04:03:03.901 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 04:03:03.902 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 04:03:03.904 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 04:03:03.905 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 04:03:03.906 Thread 'MainThread': mi